In [ ]:
# load data
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
names = ['user_id', 'item_id', 'rating', 'timestamp']
names1=['user_id','label']
#data=pd.read_csv("/content/drive/MyDrive/attacks/profiles.txt",delim_whitespace=True,names=names)
original_users_no=943 # 943 for 100k, 6040 for 1M

In [ ]:
import pickle
# with open("/content/drive/MyDrive/synthpop/new_data.pkl", "rb") as file: data=pickle.load(file)
#data=pd.read_csv("/content/drive/MyDrive/attack_datasets/Netflix300K/average/profiles_1.0.txt",delim_whitespace=True,names=names)
data=pd.read_csv("/content/drive/MyDrive/attack_datasets/Movielens100K/average/profiles_1.5.txt",delim_whitespace=True,names=names)
labels=pd.read_csv("/content/drive/MyDrive/attack_datasets/Movielens100K/average/labels_1.5.txt",delim_whitespace=True,names=names1)

In [ ]:
data.user_id.unique().shape[0]

In [ ]:
# data matrix, timestamp matrix
n_users = data.user_id.unique().shape[0]
n_items = data.item_id.unique().shape[0]
n_items = data['item_id'].max()
data_matrix = np.zeros((n_users,n_items))
#timestamp_matrix=np.zeros((n_users,n_items))
for line in data.itertuples():
    data_matrix[line[1]-1,line[2]-1] = line[3]
    #timestamp_matrix[line[1]-1,line[2]-1] = line[4]
print("Original rating matrix : ",data_matrix.shape)

Original rating matrix :  (1083, 1682)


In [ ]:
data_matrix

In [ ]:
# URIS
users=data.user_id.unique()
items=data.item_id.unique()
URIS,L,timestamps={},{},{}
for i in tqdm(users):
  URIS[i]=[]
  for j in items:
    if data_matrix[i-1][j-1]>0:
      URIS[i].append(j)
len(URIS)

100%|██████████| 3299/3299 [00:21<00:00, 156.25it/s]


3299

In [ ]:
# sorting according to timestamps
for i in tqdm(users):
  temp_arr=URIS[i]
  for j in range(len(temp_arr)):
    for k in range(0, len(temp_arr) - j - 1):
      if timestamp_matrix[i-1][temp_arr[k]-1] > timestamp_matrix[i-1][temp_arr[k+1]-1]:
        # swapping elements if elements
        # are not in the intended order
        temp = temp_arr[k]
        temp_arr[k] = temp_arr[k+1]
        temp_arr[k+1] = temp
  URIS[i]=temp_arr

100%|██████████| 3299/3299 [02:32<00:00, 21.59it/s]


In [ ]:
# preparing data for hmm
test_arr=np.array(URIS[1]).reshape(-1,1)
lengths=[len(test_arr)]
for i in tqdm(range(2,original_users_no)):
  test_arr_1=np.array(URIS[i]).reshape(-1,1)
  test_arr=np.concatenate([test_arr, test_arr_1])
  lengths.append(len(test_arr_1))

100%|██████████| 2998/2998 [00:00<00:00, 4446.71it/s]


In [ ]:
import pickle
with open(f"/content/drive/MyDrive/attacks/M1M_original_testarr.pkl", "wb") as file: pickle.dump(test_arr,file)
with open(f"/content/drive/MyDrive/attacks/M1M_original_lengths.pkl", "wb") as file: pickle.dump(lengths,file)

In [ ]:
!pip install hmmlearn

In [ ]:
from hmmlearn import hmm
from hmmlearn.hmm import MultinomialHMM
from hmmlearn.base import ConvergenceMonitor
import pickle
import pandas as pd
import numpy as np
import matplotlib as plt
from collections import defaultdict


In [ ]:
################# creating models and saving them as they take a lot of time ###################
with open(f"/content/drive/MyDrive/attacks/M1M_original_testarr.pkl", "rb") as file: test_arr=pickle.load(file)
with open(f"/content/drive/MyDrive/attacks/M1M_original_lengths.pkl", "rb") as file: lengths=pickle.load(file)
for n in tqdm(range(3,8)):
  remodel=hmm.MultinomialHMM(n_components=n,n_iter=500,algorithm='viterbi').fit(test_arr,lengths)
  with open(f"/content/drive/MyDrive/attacks/M1M_hmmmodel_{n}.pkl", "wb") as file: pickle.dump(remodel, file)

In [ ]:
# predict states
L,obsrv_prob={},{}
def predict_states():
  for i in tqdm(URIS):
    j=np.array(URIS[i]).reshape(-1,1)
    try:
      L[i]=remodel.decode(j,algorithm='viterbi')
      obsrv_prob[i]=remodel.predict_proba(j)
    except:
      print("predict error")

In [ ]:
#user matching degree
import matplotlib.pyplot as plt


UMD,umd={},[]
def user_matching_degree(i,attack,n):
  for key in tqdm(users):
    if key in L:
      state=L[key][1][0]
      #arg=start_prob[i][state]*obsrv_prob[i][0][state]
      arg=remodel.startprob_[state]*obsrv_prob[key][0][state]
      for j in range(1,len(L[key][1])):
        last_state=L[key][1][j-1]
        curr_state=L[key][1][j]
        arg1=remodel.transmat_[last_state][curr_state]*obsrv_prob[key][j][curr_state]
        arg=arg*arg1
      UMD[key]=arg**(1.0/len(L[key][1]))
      umd.append(UMD[key])
  x,y,z=[],[],[]
  for key in UMD:
    x.append(key)
    y.append(UMD[key])
    if key<944:
      z.append('royalblue')
    else:
      z.append('firebrick')

  plt.clf()
  plt.style.use('ggplot')
  plt.xlabel('User Profiles')
  plt.ylabel('User Matching Degree (UMD)')
  plt.scatter(x,y,color=z)
  plt.savefig(f'/content/drive/MyDrive/attacks/plots/umd_{i}_{attack}_attack_{n}.png')
  plt.show()
  

In [ ]:
# item entropy
import math
USD,IE,ie={},{},[]
def item_entropy():
  for key in tqdm(items):
    IE[key]=0
    col=data_matrix[:,key-1]
    prob_denom=0
    prob_num=[0,0,0,0,0]
    for j in col:
      if j>0:
        prob_denom+=1
        prob_num[int(j-1)]+=1
    for d in range(5):
      prob=prob_num[d]/prob_denom
      if prob!=0.0:
        IE[key]+=(-(prob*math.log2(prob)))
    if IE[key]!=0.0:
      ie.append(IE[key])
  print(IE)

In [ ]:
# isd
ISD={}
def item_suspicious_degree():
  umd_max=max(umd)
  umd_min=min(umd)
  ie_max=max(ie)
  ie_min=min(ie) # min(ie)
  print(ie_max,ie_min)
  for key in tqdm(items):
    UH=0
    m_norm=0
    for u in users:
      if data_matrix[u-1][key-1]>3:
        m_norm+=(1/UMD[u]-1/umd_max)/(1/umd_min-1/umd_max)
        UH+=1
    if IE[key]!=0 and UH!=0:
      ie_norm=(1/IE[key]-1/ie_max)/(1/ie_min-1/ie_max)
      ISD[key]=(m_norm/UH)*ie_norm
  print(ISD)
    # else:
    #   ISD[i]=0 # to check, entropy is 0

In [ ]:
# sdri
# sdri
SDRI,sdri={},[]
def sd_range(i,attack,n):
  for u in users:
    isd_arr=[]
    for k in URIS[u]:
      if k in ISD:
        isd_arr.append(ISD[k])
    SDRI[u]=max(isd_arr)-min(isd_arr)
    sdri.append(SDRI[u])
  sdri_max=max(sdri)
  sdri_min=min(sdri)
  print(sdri_min,sdri_max)
  x,y,z=[],[],[]
  for key in SDRI:
    x.append(key)
    y.append(SDRI[key])
    if key<944:
      z.append('royalblue')
    else:
      z.append('firebrick')

  plt.clf()
  plt.style.use('ggplot')
  plt.xlabel('User Profiles')
  plt.ylabel('Suspicious Degree Range of Items')
  plt.scatter(x,y,color=z)
  plt.savefig(f'/content/drive/MyDrive/attacks/plots/sdri_{i}_{attack}_attack_{n}.png')
  plt.show()

In [ ]:
#USD
def user_degree(i,alpha,n,attack):
  for u in tqdm(users):
    match_norm=(1/UMD[u]-1/umd_max)/(1/umd_min-1/umd_max)
    sdri_norm=(1/SDRI[u]-1/sdri_max)/(1/sdri_min-1/sdri_max)
    USD[u]=(alpha*match_norm)+(1-alpha)*sdri_norm
    usd[u-1]=USD[u]
  x,y,z=[],[],[]
  for key in USD:
    x.append(key)
    y.append(USD[key])
    if key<944:
      z.append('royalblue')
    else:
      z.append('firebrick')
  plt.clf()
  plt.style.use('ggplot')
  plt.xlabel('User Profiles')
  plt.ylabel('User Suspicious Degree (USD)')
  plt.scatter(x,y,color=z)
  plt.savefig(f'/content/drive/MyDrive/attacks/plots/usd_{i}_{attack}_attack_{alpha}_{n}.png')
  plt.show()
  return usd

In [ ]:
#algo3 clustering
from sklearn.cluster import AgglomerativeClustering
# clustering = AgglomerativeClustering(linkage='ward', n_clusters=2)
# usd=np.array(usd).reshape((-1,1))
# clustering.fit(usd)
# np.count_nonzero(clustering.labels_==1)

In [ ]:

for attack in ['random','bandwagon']:
  for i in [0.1,0.15,0.20]:
    data=pd.read_csv(f"/content/drive/MyDrive/attack_datasets/Movielens100K/{attack}/profiles_{i*10}.txt",delim_whitespace=True,names=names)
    # data matrix, timestamp matrix
    n_users = data.user_id.unique().shape[0]
    n_items = data.item_id.unique().shape[0]
    n_items = data['item_id'].max()
    data_matrix = np.zeros((n_users,n_items))
    timestamp_matrix=np.zeros((n_users,n_items))
    for line in data.itertuples():
        data_matrix[line[1]-1,line[2]-1] = line[3]
        timestamp_matrix[line[1]-1,line[2]-1] = line[4]
    print("Original rating matrix : ",data_matrix.shape)
    # URIS
    users=data.user_id.unique()
    items=data.item_id.unique()
    URIS,L,timestamps={},{},{}
    for i in tqdm(users):
      URIS[i]=[]
      for j in items:
        if data_matrix[i-1][j-1]>0:
          URIS[i].append(j)
    # sorting according to timestamps
    for i in tqdm(users):
      temp_arr=URIS[i]
      for j in range(len(temp_arr)):
        for k in range(0, len(temp_arr) - j - 1):
          if timestamp_matrix[i-1][temp_arr[k]-1] > timestamp_matrix[i-1][temp_arr[k+1]-1]:
            # swapping elements if elements
            # are not in the intended order
            temp = temp_arr[k]
            temp_arr[k] = temp_arr[k+1]
            temp_arr[k+1] = temp
      URIS[i]=temp_arr
    usd=[0]*len(users)
    final_labels=defaultdict(dict)
    for n in tqdm(range(5,6)):
      with open(f"/content/drive/MyDrive/attacks/M100K_hmmmodel_{n}.pkl", "rb") as file: remodel=pickle.load(file)
      L,obsrv_prob={},{}
      predict_states()
      UMD,umd={},[]
      user_matching_degree(i,attack,n)
      umd_max=max(umd)
      umd_min=min(umd)
      USD,IE,ie={},{},[]
      item_entropy()
      ISD={}
      item_suspicious_degree()
      SDRI,sdri={},[]
      sd_range(i,attack,n)
      sdri_max=max(sdri)
      sdri_min=min(sdri)
      for alpha in [0.9]:
        arr=user_degree(i,alpha,n,attack)
        clustering = AgglomerativeClustering(linkage='ward', n_clusters=2)
        c_arr=np.array(arr).reshape((-1,1))
        try:
          clustering.fit(c_arr)
          final_labels[n][alpha]=clustering.labels_
        except:
          print(f"{n},error")
    with open(f"/content/drive/MyDrive/attack_datasets/Movielens100K/{attack}_profiles_{i*10}_finallabels.pkl", "wb") as file: pickle.dump(final_labels, file)

In [ ]:
for attack in ['random','bandwagon']:
  print(f"-----attack-----")
  for i in [1036,1083,1130,1177]:
  #for i in [6643,6945,7247,7549]:
    print(f"-----attack size= {i*10}%--------------")
    with open(f"/content/drive/MyDrive/attack_datasets/Movielens100K/{attack}_profiles_{i*10}_finallabels.pkl", "rb") as file: results=pickle.load(file)
    # for n in range(3,6):
    n=5
    print(f"-----------n-components={n}---------------")
    # for alpha in [0.6,0.7,0.8,0.9,1.0]:
    alpha=0.9
    print(f"-------alpha={alpha}--------------")
    print(f"cluster 1 = {np.count_nonzero(results[n][alpha]==1)}")
    print(f"cluster 2= {np.count_nonzero(results[n][alpha]==0)}")
    labels=results[n][alpha]
    TP,FP,FN,precision,F1,recall=0,0,0,0,0,0
    for p in range(943):
      if labels[p]==1:FP+=1
    for p in range(943,len(labels)):
      if labels[p]==1:TP+=1
      else:FN+=1
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    F1=((2*precision)*recall)/(precision+recall)
    print(f"F1={F1}")